# Word2Vec

Word2vec produces one vector per word, whereas BoW produces one number (a wordcount). Word2vec is great for digging into documents and identifying content and subsets of content.

In [12]:
import pandas as pd

articles = pd.read_excel("data/OpArticles.xlsx")
dataset = pd.read_excel('data/OpArticles_ADUs.xlsx')

In [ ]:
!pip install gensim

In [ ]:
import re
documents = [re.sub("[^a-zA-Z]", " ", body.lower()) for body in articles["body"]]

In [3]:
from time import time
from gensim.models import Word2Vec, KeyedVectors

start_time = time()
model_articles = Word2Vec(documents, vector_size=150, window=10, min_count=2, workers=10, sg=1)

print("Training time:", time() - start_time)

Training time: 7.778205871582031


In [16]:
model_articles.wv.save("./word_vectors/model_articles")

model_articles = KeyedVectors.load("./word_vectors/model_articles")

## Pre-trained (NILC) embeddings


In [24]:
from gensim.models import KeyedVectors

# Word2Vec SKIP-GRAM 100 dimensions
model_skip_s100 = KeyedVectors.load_word2vec_format('./word_vectors/skip_s100.txt')

#FastText SKIP-GRAM 100 dimensions
ft_skip_s100 = KeyedVectors.load_word2vec_format('./word_vectors/ft_skip_s100.txt')

## Cleanup

In [13]:
import pt_core_news_sm
#nlp = pt_core_news_sm.load()
corpus = []

for i in range(0, dataset['tokens'].size):
    # get review, remove non alpha chars and convert to lower-case
    review = re.sub('[^a-zA-Z]', ' ', dataset['tokens'][i]).lower()
    #review = ' '.join([word.lemma_.lower().strip() + word.pos_ for word in nlp(review)])
    # add review to corpus
    corpus.append(review)

## Fixing the length of the input
The reviews in our corpus have variable length. However, we need to represent them with a fixed-length vector of features. One way to do it is to impose a limit on the number of word embeddings we want to include.

To convert words into their vector representations (embeddings), let's create an auxiliary function that takes in the number of embeddings we wish to include in the representation:

In [5]:
import numpy as np

def text_to_vector(embeddings, text, sequence_len):
    
    # split text into tokens
    tokens = text.split()
    
    # convert tokens to embedding vectors, up to sequence_len tokens
    vec = []
    n = 0
    i = 0
    while i < len(tokens) and n < sequence_len:   # while there are tokens and did not reach desired sequence length
        try:
            vec.extend(embeddings.get_vector(tokens[i]))
            n += 1
        except KeyError:
            True   # simply ignore out-of-vocabulary tokens
        finally:
            i += 1
    
    # add blanks up to sequence_len, if needed
    for j in range(sequence_len - n):
        vec.extend(np.zeros(embeddings.vector_size,))
    
    return vec

In [22]:
import numpy as np
from scipy import stats

lens = [len(c.split()) for c in corpus]
print(np.min(lens), np.max(lens), np.mean(lens), np.std(lens), stats.mode(lens))

1 92 16.132174640148122 10.750637692983176 ModeResult(mode=array([10]), count=array([916]))


The average length of the text spans is 16.1 tokens with standard deviation of 10.75.

## Generating training/testing sets

In this experimental setup, we will be using different word embedding models. `model_articles` was trained using the full articles dataset whereas the others were loaded and pre-trained.

In [20]:
# convert corpus into dataset with appended embeddings representation

def generate_sets(model):
    embeddings_corpus = []
    for c in corpus:
        embeddings_corpus.append(text_to_vector(model, c, 15))

    X = np.array(embeddings_corpus)
    y = dataset['label']
    return (X,y)

In [25]:
from sklearn.model_selection import train_test_split

X_articles, y_articles = generate_sets(model_skip_s100)

X_train, X_test, y_train, y_test = train_test_split(X_articles, y_articles, test_size = 0.20, random_state = 0, stratify=y_articles)


In [26]:
import sklearn.metrics as metrics
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV

sgd_model = SGDClassifier()
sgd_gs = GridSearchCV(sgd_model, n_jobs=-1, param_grid={}, cv=10, scoring="accuracy", verbose = 1)
sgd_gs.fit(X_train, y_train)
y_pred = sgd_gs.predict(X_test)

print("\nConfusion matrix:\n", metrics.confusion_matrix(y_test, y_pred))
print("Classification report:\n", metrics.classification_report(y_test, y_pred))

Fitting 10 folds for each of 1 candidates, totalling 10 fits

Confusion matrix:
 [[ 226   16  375   32   84]
 [   8   49   65    4    7]
 [ 224   47 1074   78  198]
 [  36    8  156   51   31]
 [  67   13  328   15  157]]
Classification report:
               precision    recall  f1-score   support

        Fact       0.40      0.31      0.35       733
      Policy       0.37      0.37      0.37       133
       Value       0.54      0.66      0.59      1621
    Value(+)       0.28      0.18      0.22       282
    Value(-)       0.33      0.27      0.30       580

    accuracy                           0.46      3349
   macro avg       0.38      0.36      0.37      3349
weighted avg       0.44      0.46      0.45      3349

